In [1]:
import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
# reading the data
df = pd.read_csv(r"..\data\AAPL Stock Price (15-06-204 and last 4 years).csv", 
                 parse_dates=['Date'], dayfirst=True, index_col='Date')
df.columns = df.columns.str.lower()
df.index = df.index.strftime('%Y-%m-%d')
df.shape

(1008, 5)

In [3]:
display(df.head(),df.tail(),df.sample(5))

,open,high,low,close,volume
Date,,,,,
2020-06-15,83.31,86.42,83.15,85.75,34702230
2020-06-16,87.87,88.30,86.18,88.02,41357182
2020-06-17,88.79,88.85,87.77,87.90,28601626
2020-06-18,87.85,88.36,87.31,87.93,24205096
2020-06-19,88.66,89.14,86.29,87.43,66118952


,open,high,low,close,volume
Date,,,,,
2024-06-10,196.90,197.30,192.15,193.12,97262077
2024-06-11,193.65,207.16,193.63,207.15,172373296
2024-06-12,207.37,220.20,206.90,213.07,198134293
2024-06-13,214.74,216.75,211.60,214.24,97862729
2024-06-14,213.85,215.17,211.30,212.49,70122748


,open,high,low,close,volume
Date,,,,,
2022-11-08,140.41,141.43,137.49,139.50,89908477
2022-02-07,172.86,173.95,170.95,171.66,77251204
2022-12-02,145.96,148.00,145.65,147.81,65447446
2024-02-08,189.39,189.54,187.35,188.32,40962046
2022-03-29,176.69,179.01,176.34,178.96,100589440


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1008 entries, 2020-06-15 to 2024-06-14
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    1008 non-null   float64
 1   high    1008 non-null   float64
 2   low     1008 non-null   float64
 3   close   1008 non-null   float64
 4   volume  1008 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 47.2+ KB


# Feature Engineering

In [5]:
df['moving_avg_10'] = df['close'].rolling(window=10).mean()
df['moving_avg_50'] = df['close'].rolling(window=50).mean()

In [6]:
display(df.head(),df.tail())

,open,high,low,close,volume,moving_avg_10,moving_avg_50
Date,,,,,,,
2020-06-15,83.31,86.42,83.15,85.75,34702230,NaN,NaN
2020-06-16,87.87,88.30,86.18,88.02,41357182,NaN,NaN
2020-06-17,88.79,88.85,87.77,87.90,28601626,NaN,NaN
2020-06-18,87.85,88.36,87.31,87.93,24205096,NaN,NaN
2020-06-19,88.66,89.14,86.29,87.43,66118952,NaN,NaN


,open,high,low,close,volume,moving_avg_10,moving_avg_50
Date,,,,,,,
2024-06-10,196.90,197.30,192.15,193.12,97262077,193.256,179.9666
2024-06-11,193.65,207.16,193.63,207.15,172373296,194.972,180.7090
2024-06-12,207.37,220.20,206.90,213.07,198134293,197.250,181.5936
2024-06-13,214.74,216.75,211.60,214.24,97862729,199.545,182.4854
2024-06-14,213.85,215.17,211.30,212.49,70122748,201.569,183.3588


In [7]:
display(f"Shape of data before dropping null values: {df.shape}")
df = df.dropna()
display(f"Shape of data after dropping null values: {df.shape}")

'Shape of data before dropping null values: (1008, 7)'

'Shape of data after dropping null values: (959, 7)'

# Model Selection

In [8]:
# treating moving average as indepedent variable and closing price as target variable
x = df[['moving_avg_10', 'moving_avg_50']]
y = df['close']

# splitting the data into train and test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [9]:
LinearRegression()

LinearRegression()

In [10]:
def best_model_gridsearch(independent_variables: pd.DataFrame, 
                          dependent_variable: pd.DataFrame) -> pd.DataFrame:
    """This function performs a grid search for the best model among several machine learning algorithms.

    Parameters:
    - independent_variables (pd.DataFrame): A DataFrame containing the independent variables used for prediction.
    - dependent_variable (pd.DataFrame): A DataFrame containing the dependent variable to be predicted.

    Returns:
    - pd.DataFrame: A DataFrame containing the best score achieved by each model, the best parameters used for each model, and the name of the model.
    """
    algorithms = {
        'LinearRegression': {
            'model': LinearRegression(),
            'params': {
                'fit_intercept': [True, False],
                'copy_X': [True, False],
                'n_jobs': [None, 1, 2, 3],
                'positive': [True, False]
            }
        },
        
        'Ridge': {
            'model': Ridge(),
            'params': {
                'alpha': [0.1,1,10,100],
                'fit_intercept': [True, False],
                'copy_X': [True, False],
                'solver': ['auto','svd','cholesky','lsqr', 'sparse_cg', 'sag','saga'],
                'positive': [True, False]
            }
        },
        
        'Lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [0.1,1,10,100],
                'fit_intercept': [True, False],
                'copy_X': [True, False],
                'precompute': [True, False],
                'warm_start':[True, False],
                'selection': ['cyclic', 'random'],
                'positive': [True, False]
            }
        },
        
        'DecisionTree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion': ['squared_error','friedman_mse','absolute_error', 'poisson'],
                'splitter': ['best','random'],
                'max_depth': [None, 10, 20, 30]
            }
        },
        
        'RandomForest': {
            'model': RandomForestRegressor(),
            'params': {
                'n_estimators': [50, 100, 200],
                'max_depth': [None, 10, 20],
                'bootstrap': [True, False],
                'criterion': ['mse', 'mae', 'poisson']
            }
        },
        
        'SVR': {
            'model': SVR(),
            'params': {
                'C': [0.1, 1, 10],
                'gamma': ['scale', 'auto'],
                'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
            }
        }
    }
    
    scores = []
    cross_validation = ShuffleSplit(n_splits= 5, test_size= 0.2, random_state= 42)
    
    for algorithm, config in tqdm(algorithms.items(), desc='Grid Searching Models'):
        grid_search = GridSearchCV(config['model'],config['params'],cv=cross_validation,
                                  scoring='neg_mean_squared_error',
                                  return_train_score=False)
        grid_search.fit(independent_variables,dependent_variable)
        scores.append({
            'model': algorithm,
            'best_score': grid_search.best_score_,
            'best_params': grid_search.best_params_
        })
        
    return pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])

In [ ]:
best_model_gridsearch(independent_variables=x, dependent_variable=y)